In [3]:
import pandas as pd
import os


ratings = pd.read_csv('../data/BX-Book-Ratings-cleaned.csv', sep=';', converters = {'user': int, 'isbn': str, 'rating': float} , encoding="CP1252")
books = pd.read_csv(os.path.abspath('') + '/../backend/app/books/MostLeastPopular.csv', sep=",", encoding="CP1252")
books = books.values.tolist()
user_ratings = {}
grouped = ratings.groupby('user')
# take second element for sort
def takeSecond(elem):
    return elem[1]

for user, g_rate in grouped:
    vals = list(map(lambda x: [x[1], x[2]],g_rate.values.tolist()))
    vals = list(filter(lambda x: x[1] > 0,vals))
    vals.sort(key=takeSecond, reverse=True)
    meanrating = 0
    if(len(vals)>0):
        meanrating = sum(list(map(lambda x: x[1], vals)))/len(vals)
    amountrel = len(list(filter(lambda x: x[1] > meanrating, vals) ))
    user_ratings[user] = [vals, meanrating, amountrel]

total_prec = 0
total_recall = 0

for user, item in user_ratings.items():
    ratings = item[0]
    mean = item[1]
    totalrel = item[2]
    tp = 0
    fp = 0
    for rating in ratings:
        for book in books:
            if book[0] == rating[0] and rating[1]>mean:
                tp += 1
            if book[0] == rating[0] and rating[1]<=mean:
                fp += 1
    total_prec += tp/len(books)

    if(totalrel > 0):
        total_recall += tp/totalrel

print("len: ", len(books))
print("Precision: ", total_prec/len(user_ratings))
print("Recall: ", total_recall/len(user_ratings))


len:  65
Precision:  0.0014736855769637218
Recall:  0.01508688529585272


In [4]:
import math
total_prec = 0
total_recall = 0

totalndcg = 0


usertps = {}

for user, item in list(user_ratings.items()):
    ratings = item[0]
    tp = []
    for rating in ratings:
        for idx, book in enumerate(books):
            if book[0] == rating[0] and rating[1]>mean:
                tp.append([rating, idx])
    if(len(ratings) > 1):
        usertps[user] = tp

        
test = {
    "a": [
        [['059035342X', 10.0], 2],
        [['059035342X', 10.0], 3],
        [['059035342X', 10.0], 4],
    ]
}


totalndcg = []
for user, tplist in usertps.items():
    dcg = 0
    idcg = 0
    for ind, item in enumerate(tplist):
        if(item[1] >= 2):
            dcg += 1/math.log2(item[1])
    for ind in range(1, len(tplist)):
        idcg += 1/math.log2(ind+1)

    if(len(tplist)>0):
        idcg += 1
        totalndcg.append(dcg/idcg)
        
to = 0
for item in totalndcg:
    to += item

if(len(totalndcg) > 0):
    print("ndcg", to/len(totalndcg))
else:
    print("no tps")



ndcg 0.2610353210675939
